In [1]:
%load_ext autoreload
%autoreload 2
!source /home/murilo/RelNetCare/.env

In [2]:
# import hunspell
import re
import torch
import language_tool_python
from transformers import pipeline

def extract_bot_reply(outputs, bot_name='Adele'):
    # Extracts raw response from full generated text
    raw_response = outputs[0]["generated_text"].split('<|assistant|>\n')[-1]

    # Define the pattern to extract the bot's reply
    pattern = rf'{bot_name}: ([^\(\n]*)'

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, raw_response)

    # Return the first match stripped of leading/trailing whitespace, or None if no match
    return matches[0].strip() if matches else None


def correct_text(input_text, language='de-DE'):
    # Initialize the LanguageTool object with the specified language
    tool = language_tool_python.LanguageTool(language)

    # Check the text
    matches = tool.check(input_text)

    # Generate corrected text
    corrected_text = language_tool_python.utils.correct(input_text, matches)

    return corrected_text


if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

print(f"### device={device} ###")


if 'pipe' in locals():
    print('Empting gpu cache...')
    del pipe  # Replace 'model' with the name of your existing model variable
    torch.cuda.empty_cache()  # Clear the cache


# pipe = pipeline("text-generation", model="Open-Orca/Mistral-7B-OpenOrca", torch_dtype=torch.bfloat16, device_map=device)
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map=device)


2023-11-22 13:58:14.616808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 13:58:14.616858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 13:58:14.616890: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 13:58:14.631713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 13:58:15.564203: W tensorflow/c

### device=cuda:0 ###


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
# # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
# messages = [
#     {
#         "role": "system",
#         "content": prompt },
#         {"role": "assistant",
#          "content": "Adele: Guten Abend, Hilde! Hier spricht Adele. Hast du etwas Zeit für ein kurzes Gespräch? Wie war dein Tag?"},
#         {"role": "user",
#          "content": "Hilde: Ja, ich bin heute Morgen in guter Stimmung."},
    
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.3
#                , top_k=50, top_p=0.95
#                )
# print(outputs[0]["generated_text"])

# # Example usage
# response = extract_bot_reply(outputs, bot_name='Adele')
# print(response)

In [29]:
# few_shot = """
# Beispiele für Ausgangsszenarien:

# # Offen für Gespräche:
# {bot_name}: Guten Tag, Jeniffer! Hier ist {bot_name}. Können wir über jemanden sprechen, den du magst?
# Jeniffer: Ja, Robert ist sehr lieb zu mir.
# {bot_name}: Das klingt wunderbar, Jeniffer. Wie lange kennt ihr euch schon?

# # Ablehnungen:
# {bot_name}: Hallo Jeniffer, hier ist {bot_name}. Hast du Zeit für ein Gespräch?
# Jeniffer: Nein, heute nicht...
# {bot_name}: Verstehe, Jeniffer. Ich bin da, wenn du reden willst. Einen schönen Tag noch!

# # Verwirrt:
# {bot_name}: Hallo Jeniffer, hier ist {bot_name}! Können wir über dein Wohlbefinden sprechen?
# Jeniffer: Wer bist du?
# {bot_name}: Ich bin {bot_name}, deine Gesundheitsassistentin. Ich bin hier, um dir zuzuhören und dir zu helfen.
# """.format(user_name='Hilde', bot_name='Adele', few_shot=few_shot)


# few_shot = ""

few_shot = """
Beispiele für Ausgangsszenarien:

{bot_name}: Guten Tag, Jeniffer! Hier ist {bot_name}. Können wir über jemanden sprechen, den du magst?
Jeniffer: Ja, Robert ist sehr lieb zu mir.
{bot_name}: Das klingt wunderbar, Jeniffer. Wie lange kennt ihr euch schon?
""".format(user_name='Hilde', bot_name='Adele', few_shot=few_shot)


prompt = """
Du bist eine KI namens '{bot_name}', deine Aufgabe ist es, nett zu sein!

Du interagierst mit dem Benutzer '{user_name}', einer älteren Person.
{few_shot}
Verwende IMMER Du ANSTELLE VON Sie, wenn du dich auf '{user_name}' beziehst.

Verwende IMMER den Namen des Benutzers '{user_name}'.

Halte deine Antworten immer unter 20 Wörtern, also kurz.
""".format(user_name='Hilde', bot_name='Adele', few_shot=few_shot)

In [30]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display

def extract_bot_reply(outputs, bot_name='Adele'):
    # Extracts raw response from full generated text
    
    raw_text = outputs[0]["generated_text"]

    patterns = ('<|im_start|>assistant\n', '<|assistant|>\n')
    
    for p in patterns:
        if p in raw_text:
            break
    
    raw_response = outputs[0]["generated_text"].split(p)[-1]

    # Define the pattern to extract the bot's reply
    pattern = rf'{bot_name}: ([^\(\n]*)'

    # Find all matches of the pattern in the text
    matches = re.findall(pattern, raw_response)

    # Return the first match stripped of leading/trailing whitespace, or None if no match
    return matches[0].strip() if matches else raw_response

# Initialize the conversation history
messages = [
    {
        "role": "system",
        "content": prompt
    },
    {
        "role": "assistant",
        "content": "Adele: Guten Abend, Hilde! Hier spricht Adele. Hast du etwas Zeit für ein kurzes Gespräch? Wie war dein Tag?"
    }
]

for m in messages:
    print(m.get('content'))

while True:

    # Get user input
    user_input = input("Hilde: ")

    display("Hilde: " + user_input)

    # Break the loop if the user says something like 'exit', 'quit', etc.
    if user_input.lower() in ['exit', 'quit', 'end']:
        break

    # Update the messages list with the new user input and bot response
    messages.extend([
        {"role": "user", "content": f"Hilde: {user_input}"},
    ])
    
    # Format the prompt using the chat template
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate the bot's response
    outputs = pipe(prompt, max_new_tokens=128, 
                   do_sample=True, temperature=0.3, top_k=50, top_p=0.95
                   )
    bot_response = extract_bot_reply(outputs, bot_name='Adele')

    messages.extend([
        {"role": "assistant", "content": f"Adele: {bot_response}"}
    ])
    
    display("Adele: " + bot_response)





Du bist eine KI namens 'Adele', deine Aufgabe ist es, nett zu sein!

Du interagierst mit dem Benutzer 'Hilde', einer älteren Person.

Beispiele für Ausgangsszenarien:

{bot_name}: Guten Tag, Jeniffer! Hier ist {bot_name}. Können wir über jemanden sprechen, den du magst?
Jeniffer: Ja, Robert ist sehr lieb zu mir.
{bot_name}: Das klingt wunderbar, Jeniffer. Wie lange kennt ihr euch schon?

Verwende IMMER Du ANSTELLE VON Sie, wenn du dich auf 'Hilde' beziehst.

Verwende IMMER den Namen des Benutzers 'Hilde'.

Halte deine Antworten immer unter 20 Wörtern, also kurz.

Adele: Guten Abend, Hilde! Hier spricht Adele. Hast du etwas Zeit für ein kurzes Gespräch? Wie war dein Tag?


'Hilde: war schön soweit'

'Adele: Glücklich sehen, Hilde! Was macht dir Spaß am Abend? Lesen, Fernsehen oder Spiele?'

'Hilde: exit'

SyntaxError: incomplete input (3939930801.py, line 29)